# 예제 3.53-3.56: 이진 분류 (Binary Classification)

## 학습목표
1. **이진 분류(Binary Classification)** 개념 이해하기 - 두 클래스 중 하나로 분류
2. **Sigmoid 활성화 함수** 사용법 익히기 - 출력을 0~1 사이로 변환
3. **BCELoss (Binary Cross Entropy)** 손실함수 이해하기
4. **Train/Validation/Test 분할** 적용하기
5. **분류 임계값(Threshold)** 활용하기

---

#### 예제 3.53 라이브러리 임포트

**이진 분류 vs 회귀**
- 회귀: 연속적인 값 예측 (예: 가격, 온도)
- 이진 분류: 0 또는 1로 분류 (예: 스팸/정상, 합격/불합격)

In [ ]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split

---

#### 예제 3.54 커스텀 데이터셋 클래스

3개의 입력 특성(x1, x2, x3)으로 이진 레이블(y)을 예측

In [ ]:
class CustomDataset(Dataset):
    """이진 분류용 커스텀 데이터셋"""
    
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        # 3개의 입력 특성
        self.x1 = df.iloc[:, 0].values
        self.x2 = df.iloc[:, 1].values
        self.x3 = df.iloc[:, 2].values
        # 이진 레이블 (0 또는 1)
        self.y = df.iloc[:, 3].values
        self.length = len(df)

    def __getitem__(self, index):
        # 입력: 3개 특성을 하나의 텐서로
        x = torch.FloatTensor([self.x1[index], self.x2[index], self.x3[index]])
        # 출력: 이진 레이블 (0.0 또는 1.0)
        y = torch.FloatTensor([int(self.y[index])])
        return x, y

    def __len__(self):
        return self.length

---

#### 커스텀 모델 클래스

**Sigmoid 활성화 함수**
- 출력을 0~1 사이로 변환
- 이진 분류에서 확률로 해석 가능
- σ(x) = 1 / (1 + e^(-x))

In [ ]:
class CustomModel(nn.Module):
    """이진 분류 모델"""
    
    def __init__(self):
        super().__init__()
        # nn.Sequential: 여러 레이어를 순차적으로 연결
        self.layer = nn.Sequential(
            nn.Linear(3, 1),  # 입력 3 → 출력 1
            nn.Sigmoid()       # 출력을 0~1 사이로 변환
        )

    def forward(self, x):
        x = self.layer(x)
        return x

---

#### 예제 3.55 데이터 분할 및 데이터로더

Train(80%) / Validation(10%) / Test(10%)로 분할

In [ ]:
# 데이터셋 생성 및 분할
dataset = CustomDataset("../datasets/binary.csv")
dataset_size = len(dataset)

# 80% 훈련, 10% 검증, 10% 테스트
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

# random_split으로 데이터 분할 (재현성을 위해 시드 고정)
train_dataset, validation_dataset, test_dataset = random_split(
    dataset, 
    [train_size, validation_size, test_size], 
    torch.manual_seed(4)
)

# 각 데이터셋용 데이터로더 생성
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

print(f"훈련 데이터: {train_size}개, 검증 데이터: {validation_size}개, 테스트 데이터: {test_size}개")

---

#### 모델, 손실함수, 옵티마이저 정의

**BCELoss (Binary Cross Entropy Loss)**
- 이진 분류용 손실함수
- 예측값(0~1)과 실제 레이블(0 또는 1) 간의 차이 측정

In [ ]:
# GPU 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

# 모델 생성
model = CustomModel().to(device)

# BCELoss: Binary Cross Entropy Loss (이진 분류용)
criterion = nn.BCELoss().to(device)

# 옵티마이저
optimizer = optim.SGD(model.parameters(), lr=0.0001)

---

#### 학습 루프

In [ ]:
# 학습 루프
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)          # 순전파 (0~1 사이 값 출력)
        loss = criterion(output, y)  # BCE 손실 계산

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

---

#### 예제 3.56 검증 데이터로 평가

**분류 임계값(Threshold)**
- 일반적으로 0.5 사용
- output >= 0.5 → 클래스 1로 분류
- output < 0.5 → 클래스 0으로 분류

In [ ]:
# 검증 데이터로 평가
with torch.no_grad():  # 기울기 계산 비활성화
    model.eval()  # 평가 모드
    
    for x, y in validation_dataloader:
        x = x.to(device)
        y = y.to(device)
        
        outputs = model(x)  # 0~1 사이 확률값
        
        print("예측 확률:", outputs.cpu().numpy().flatten())
        # 임계값 0.5로 분류: True면 1, False면 0
        print("예측 클래스:", (outputs >= 0.5).cpu().numpy().flatten())
        print("실제 레이블:", y.cpu().numpy().flatten())
        print("--------------------")